In [1]:
import pyodbc
import pandas as pd
import numpy as np
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip


## Setup Python SQL Server Connection

In [44]:
#Setup Python SQL Server Connection
cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=.;"
            "Database=inr-docker-primary;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
#Run an SQL Query
cursor = cnxn.cursor()

In [4]:
FileInformation = pd.read_sql("SELECT * FROM FileInformation", cnxn)
VideoSegment = pd.read_sql("SELECT * FROM VideoSegment", cnxn)

In [5]:
VideoSegment.head()

,Id,PatientTaskHandMappingId,SegmentId,Start,End,CreatedAt,UpdatedAt
0,1,1,1,75,92,2022-11-29 06:34:16.325452,2023-02-07 10:31:38.315948
1,2,1,2,92,111,2022-11-29 06:34:16.392138,2023-02-07 10:31:38.333300
2,3,1,3,111,139,2022-11-29 06:34:16.400686,2023-02-07 10:31:38.341215
3,4,1,4,138,169,2022-11-29 06:34:16.408713,2023-02-07 10:31:38.349319
4,5,9,1,65,99,2022-11-29 07:34:04.465755,2022-12-01 00:26:01.839106


In [6]:
Segment = pd.read_sql("SELECT * FROM Segment", cnxn)
Segment

,Id,Name
0,1,IP
1,2,T
2,3,MTR1
3,4,PR
4,5,MTR2
5,6,TG
6,7,GI
7,9,GT


In [33]:
dic = {}
for i in np.arange(Segment.shape[0]):
    seg = Segment.iloc[i]
    dic[seg.Id]=seg.Name
dic[7]='GIP'
dic

{1: 'IP', 2: 'T', 3: 'MTR1', 4: 'PR', 5: 'MTR2', 6: 'TG', 7: 'GIP', 9: 'GT'}

In [13]:
newfolder = '..\\public\\TrimmedVideos\\'
folder = '..\\public\\Videos\\'
vid=cv2.VideoCapture(folder+'ARAT_02_left_Impaired_cam2_activity18 .mp4')
VideoFileClip(folder+'ARAT_02_left_Impaired_cam2_activity18 .mp4')
if (vid.isOpened()

OSError: MoviePy error: failed to read the duration of file ..\public\Videos\ARAT_02_left_Impaired_cam2_activity18 .mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ff

In [29]:
clip.duration, start_time, end_time
subclip = clip.subclip(89, clip.duration)
clip.duration, start_frame / clip.fps, end_frame / clip.fps
ffmpeg_extract_subclip(folder+video, 89, clip.duration, targetname=newfolder+dic[segId]+"_"+video)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


In [30]:
newfolder = '..\\public\\TrimmedVideos\\'
folder = '..\\public\\Videos\\'
corrupted_videos = []
df = pd.DataFrame()
for i in np.arange(VideoSegment.shape[0]):
    vs = VideoSegment.iloc[i]
    pthId = vs.PatientTaskHandMappingId
    segId = vs.SegmentId
    start_frame = vs.Start
    end_frame = vs.End
    fi = FileInformation[FileInformation.PatientTaskHandmappingId == pthId]
    for j in np.arange(fi.shape[0]):
        file = fi.iloc[j]
        video = file.FileName
        camId = file.CameraId
        # Load the video file
        vid=cv2.VideoCapture(folder+video)
        if (vid.isOpened()):
            clip = VideoFileClip(folder+video)
            if (clip.duration):
                # Calculate the start and end times in seconds
                start_time = start_frame / clip.fps
                end_time = end_frame / clip.fps
                if (clip.duration < end_time):
                    end_time = clip.duration
                if (start_time >= end_time):
                    start_time = end_time - 1
                # Extract the subclip
                subclip = clip.subclip(start_time, end_time)
                # Save the subclip to a new file
                ffmpeg_extract_subclip(folder+video, start_time, end_time, targetname=newfolder+dic[segId]+"_"+video)

                row = {'SegmentId': segId, 
                       'PatientTaskHandMappingId': pthId, 
                       'CameraId': camId, 
                       'FileName': dic[segId] + "_" + video,
                       'TimeDuration': end_time-start_time
                      }
                df = df.append(row, ignore_index = True)
            else: #corrupted file
                corrupted_videos.append(video)
        else: #corrupted file
            corrupted_videos.append(video)
df

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

...

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

,CameraId,FileName,PatientTaskHandMappingId,SegmentId,TimeDuration
0,2.0,IP_ARAT_02_left_Impaired_cam2_activity2.mp4,1.0,1.0,0.566667
1,1.0,IP_ARAT_02_left_Impaired_cam1_activity2.mp4,1.0,1.0,0.566667
2,4.0,IP_ARAT_02_left_Impaired_cam4_activity2.mp4,1.0,1.0,0.566667
3,3.0,IP_ARAT_02_left_Impaired_cam3_activity2.mp4,1.0,1.0,0.566667
4,2.0,T_ARAT_02_left_Impaired_cam2_activity2.mp4,1.0,2.0,0.633333
...,...,...,...,...,...
2919,4.0,MTR1_ARAT_013_left_Impaired_cam4_activity15.mp4,236.0,3.0,6.700000
2920,3.0,PR_ARAT_013_left_Impaired_cam3_activity15.mp4,236.0,4.0,1.000000
2921,2.0,PR_ARAT_013_left_Impaired_cam2_activity15.mp4,236.0,4.0,1.000000
2922,1.0,PR_ARAT_013_left_Impaired_cam1_activity15.mp4,236.0,4.0,1.000000


In [45]:
insert_query = '''SET IDENTITY_INSERT SegmentFileInformation ON
INSERT INTO SegmentFileInformation (Id, FileName, PatientTaskHandMappingId, CameraId, SegmentId, TimeDuration)
                  VALUES (?, ?, ?, ?, ?, ?)'''
for i in np.arange(df.shape[0]):
        row = df.iloc[i]
        print(row.TimeDuration)
        values = (
            int(i+1), 
            row.FileName, 
            int(row.PatientTaskHandMappingId), 
            int(row.CameraId), 
            int(row.SegmentId), 
            row.TimeDuration)
        
        cursor.execute(insert_query, values)

0.5666666666666669
0.5666666666666669
0.5666666666666669
0.5666666666666669
0.6333333333333333
0.6333333333333333
0.6333333333333333
0.6333333333333333
0.9333333333333336
0.9333333333333336
0.9333333333333336
0.9333333333333336
1.033333333333334
1.033333333333334
1.033333333333334
1.033333333333334
1.1333333333333333
1.1333333333333333
1.1333333333333333
1.1333333333333333
0.43333333333333357
0.43333333333333357
0.43333333333333357
0.43333333333333357
1.5999999999999996
1.5999999999999996
1.5999999999999996
1.5999999999999996
0.8333333333333339
0.8333333333333339
0.8333333333333339
0.8333333333333339
0.6000000000000001
0.6000000000000001
0.6000000000000001
0.6000000000000001
1.1333333333333329
1.1333333333333329
1.1333333333333329
1.1333333333333329
1.4333333333333327
1.4333333333333327
1.4333333333333327
1.4333333333333327
1.4666666666666668
1.4666666666666668
1.4666666666666668
1.4666666666666668
0.7666666666666666
0.7666666666666666
0.7666666666666666
0.7666666666666666
0.5
0.5
0.5


0.6000000000000001
0.6000000000000001
0.6000000000000001
1.4333333333333331
1.4333333333333331
1.4333333333333331
1.4333333333333331
1.1333333333333337
1.1333333333333337
1.1333333333333337
1.1333333333333337
4.533333333333332
4.533333333333332
4.533333333333332
4.533333333333332
1.0333333333333332
1.0333333333333332
1.0333333333333332
1.0333333333333332
3.3000000000000007
3.3000000000000007
3.3000000000000007
3.3000000000000007
0.6333333333333333
0.6333333333333333
0.6333333333333333
0.6333333333333333
2.8333333333333335
2.8333333333333335
2.8333333333333335
2.8333333333333335
0.666666666666667
0.666666666666667
0.666666666666667
0.666666666666667
2.466666666666666
2.466666666666666
2.466666666666666
2.466666666666666
0.5666666666666664
0.5666666666666664
0.5666666666666664
0.5666666666666664
0.5000000000000004
0.5000000000000004
0.5000000000000004
0.5000000000000004
0.2999999999999998
0.2999999999999998
0.2999999999999998
0.2999999999999998
0.40000000000000013
0.40000000000000013
0.4

0.7
0.7
0.7
3.5000000000000004
3.5000000000000004
3.5000000000000004
3.5000000000000004
1.0999999999999996
1.0999999999999996
1.0999999999999996
1.0999999999999996
0.9333333333333336
0.9333333333333336
0.9333333333333336
0.9333333333333336
0.9000000000000001
0.9000000000000001
0.9000000000000001
0.9000000000000001
1.8000000000000003
1.8000000000000003
1.8000000000000003
1.8000000000000003
1.7666666666666666
1.7666666666666666
1.7666666666666666
1.7666666666666666
0.4333333333333327
0.4333333333333327
0.4333333333333327
0.4333333333333327
0.7000000000000002
0.7000000000000002
0.7000000000000002
0.7000000000000002
0.8999999999999999
0.8999999999999999
0.8999999999999999
0.8999999999999999
0.6999999999999997
0.6999999999999997
0.6999999999999997
0.6999999999999997
1.4333333333333336
1.4333333333333336
1.4333333333333336
1.4333333333333336
0.8999999999999999
0.8999999999999999
0.8999999999999999
0.8999999999999999
2.5
2.5
2.5
2.5
1.5999999999999996
1.5999999999999996
1.5999999999999996
1.5

1.0333333333333332
1.0333333333333332
1.0333333333333332
1.0333333333333332
2.5333333333333337
2.5333333333333337
2.5333333333333337
2.5333333333333337
0.5
0.5
0.5
0.5
0.5666666666666664
0.5666666666666664
0.5666666666666664
0.5666666666666664
7.833333333333333
7.833333333333333
7.833333333333333
7.833333333333333
40.233333333333334
40.233333333333334
40.233333333333334
40.233333333333334
19.0
19.0
19.0
19.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.7000000000000002
0.7000000000000002
0.7000000000000002
0.7000000000000002
1.3999999999999997
1.3999999999999997
1.3999999999999997
1.3999999999999997
0.8333333333333335
0.8333333333333335
0.8333333333333335
0.8333333333333335
1.1333333333333337
1.1333333333333337
1.1333333333333337
1.1333333333333337
3.2
3.2
3.2
3.2
45.266666666666666
45.266666666666666
45.266666666666666
45.266666666666666
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5000000000000002
0.5000000000000002
0.5000000000000002
0.5000000000000002
0.833333333333333
0.833333333333333


0.7999999999999998
0.8333333333333335
0.8333333333333335
0.8333333333333335
0.8333333333333335
0.8666666666666667
0.8666666666666667
0.8666666666666667
0.8666666666666667
0.766666666666667
0.766666666666667
0.766666666666667
0.766666666666667
0.33333333333333326
0.33333333333333326
0.33333333333333326
0.33333333333333326
0.6333333333333333
0.6333333333333333
0.6333333333333333
0.6333333333333333
4.6
4.6
4.6
4.6
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.2000000000000002
1.2000000000000002
1.13
1.2000000000000002
1.0000000000000002
1.0000000000000002
1.0000000000000002
1.0000000000000002
33.5
33.5
33.5
33.5
1.0999999999999943
1.0999999999999943
1.0999999999999943
1.0999999999999943
0.6333333333333329
0.6333333333333329
0.6333333333333329
0.6333333333333329
0.7666666666666667
0.7666666666666667
0.7666666666666667
0.7666666666666667
1.0333333333333334
1.0333333333333334
1.0333333333333334
1.0333333333333334
1.1666666666666665
1.1666666666666665
1.1666666666666665
1.1666666666666665
1.1999999999999

In [34]:
df

,CameraId,FileName,PatientTaskHandMappingId,SegmentId,TimeDuration
0,2.0,IP_ARAT_02_left_Impaired_cam2_activity2.mp4,1.0,1.0,0.566667
1,1.0,IP_ARAT_02_left_Impaired_cam1_activity2.mp4,1.0,1.0,0.566667
2,4.0,IP_ARAT_02_left_Impaired_cam4_activity2.mp4,1.0,1.0,0.566667
3,3.0,IP_ARAT_02_left_Impaired_cam3_activity2.mp4,1.0,1.0,0.566667
4,2.0,T_ARAT_02_left_Impaired_cam2_activity2.mp4,1.0,2.0,0.633333
...,...,...,...,...,...
2919,4.0,MTR1_ARAT_013_left_Impaired_cam4_activity15.mp4,236.0,3.0,6.700000
2920,3.0,PR_ARAT_013_left_Impaired_cam3_activity15.mp4,236.0,4.0,1.000000
2921,2.0,PR_ARAT_013_left_Impaired_cam2_activity15.mp4,236.0,4.0,1.000000
2922,1.0,PR_ARAT_013_left_Impaired_cam1_activity15.mp4,236.0,4.0,1.000000


In [46]:
cnxn.commit()
cursor.close()
cnxn.close()